In [1]:
from time import time
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from PIL import Image
from keras import optimizers
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate,Add
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

/home/hsun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_img = np.load("../train_img_aug.npy").reshape([-1,256,256,1])
train_seg = np.load("../train_seg_aug.npy").reshape([-1,256,256,1])
train_flo = np.load("../train_flo_aug.npy")
train_rflo = np.load("../train_rflo_aug.npy")
#train_img = np.log(train_img+1.0)
train_seg = np_utils.to_categorical(train_seg, 2)

test_img = np.load("../test_img.npy").reshape([-1,256,256,1])
test_seg = np.load("../test_seg.npy").reshape([-1,256,256,1])
test_flo = np.load("../test_flo.npy")
test_rflo = np.load("../test_rflo.npy")
test_seg = np_utils.to_categorical(test_seg, 2)

In [17]:
random.seed(23)

In [18]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:
input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(input_img)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Concatenate()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Concatenate()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 =Concatenate()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Concatenate()([conv1,up9,input_flo,input_rflo])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

unet = Model(input = [input_img,input_flo,input_rflo], output = conv10)

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
#early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='auto')
csv_logger = CSVLogger('last.csv')

#decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('last.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#autoencoder = Model(inputs=input_img, outputs=decoded)
unet.compile(optimizer=adam, loss=dice_loss)

/home/hsun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`


In [19]:
unet.fit([train_img,train_flo,train_rflo], train_seg,
        shuffle=True,
        epochs=200,
        batch_size=20,
        validation_data=([test_img,test_flo,test_rflo],test_seg),callbacks=[EarlyStopping,checkpoint,csv_logger])

Train on 13800 samples, validate on 127 samples
Epoch 1/200
13800/13800 [==============================] - 914s 66ms/step - loss: -0.4533 - val_loss: -0.4953

Epoch 00001: val_loss improved from inf to -0.49532, saving model to last.h5
Epoch 2/200
13800/13800 [==============================] - 910s 66ms/step - loss: -0.6285 - val_loss: -0.6830

Epoch 00002: val_loss improved from -0.49532 to -0.68304, saving model to last.h5
Epoch 3/200
13800/13800 [==============================] - 907s 66ms/step - loss: -0.7708 - val_loss: -0.7368

Epoch 00003: val_loss improved from -0.68304 to -0.73684, saving model to last.h5
Epoch 4/200
13800/13800 [==============================] - 908s 66ms/step - loss: -0.8358 - val_loss: -0.7530

Epoch 00004: val_loss improved from -0.73684 to -0.75299, saving model to last.h5
Epoch 5/200
13800/13800 [==============================] - 907s 66ms/step - loss: -0.8655 - val_loss: -0.7718

Epoch 00005: val_loss improved from -0.75299 to -0.77179, saving model to 